In [20]:
import pandas as pd
import dotenv
import numpy as np
import os
import asyncio
import nest_asyncio
from tmdbv3api import TMDb, Movie, Person, Genre
from tqdm.asyncio import tqdm
import gc

dotenv.load_dotenv()
nest_asyncio.apply()
gc.collect()

tmdb = TMDb()
tmdb.api_key = os.environ.get("TMDB_API_KEY")

pd.options.display.max_columns = None
pd.options.display.max_rows = None

movie = Movie()
person = Person()
genre = Genre()

In [9]:
movie_title_map = {
        'Lamb':'Dýrið',
        'Raw':'Grave',
        'Star Wars':'Star Wars ' ## TEMPORARY BUG?
        }

In [10]:
def genre_dict_create():
    genre_dict = {}
    for i in genre.movie_list():
        genre_dict[i.id] = i.name
    return(genre_dict)

def valid_year(result, year):
    """Helper to validate release year with a +/- 1 tolerance."""
    try:
        release_year = int(getattr(result, 'release_date', '')[:4]) if getattr(result, 'release_date', None) else None
        return release_year in {year, year - 1, year + 1}
    except (TypeError, ValueError):
        return False

def process_crew(crew):
    """Process director, producer, writer, and other crew details."""
    def crew_info(job):
        return [c for c in crew if c['job'] == job]

    directors = crew_info('Director')
    producers = crew_info('Producer')

    return {
        'Directors': [d.name for d in directors],
        'Producers': [p.name for p in producers],
    }

def process_cast(cast, cast_n):
    """Process the top N cast members."""
    cast_info = list(cast)[:cast_n]
    return {
        'Top_Cast': [c.name for c in cast_info],
        'Top_Cast_Popularity_Avg': np.mean([c.popularity for c in cast_info]) if cast_info else None,
    }

In [11]:
async def movie_search_async(title, year, genres, cast_n=10):
    """
    Async parallelized version of movie search.
    """
    not_found = []
    search = await asyncio.to_thread(movie.search, title)

    res = next((result for result in search if valid_year(result, year)), None)

    if res is None:
        not_found.append(title)
        return not_found

    return await extract_movie_data_async(res, genres, cast_n)

async def extract_movie_data_async(res, genres, cast_n):
    """Extract movie data using async tasks."""
    movie_id = res.id

    details_task = asyncio.to_thread(Movie(movie_id).details, movie_id)
    credits_task = asyncio.to_thread(Movie(movie_id).credits, movie_id)

    movie_details, credits = await asyncio.gather(details_task, credits_task)

    data = {
        'Movie_ID': movie_id,
        'Title': res.title,
        'Genres': [genres.get(g) for g in res.genre_ids],
        'Language': res.original_language,
        'Overview': movie_details.overview,
        'Popularity': movie_details.popularity,
        'Release_Date': movie_details.release_date,
        'Vote_Avg': movie_details.vote_average,
        'Budget': movie_details.budget,
        'Revenue': movie_details.revenue,
        'Runtime': movie_details.runtime,
        'Keywords': [k.name for k in movie_details.keywords.keywords],
    }

    data.update(process_crew(credits.crew))
    data.update(process_cast(credits.cast, cast_n))

    return data

async def fill_df_async(df, genres):
    """Async function to process multiple movies in parallel."""
    tasks = [movie_search_async(row['Name'], row['Year'], genres) for _, row in df.iterrows()]
    results = []
    for task in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Processing Movies"):
        results.append(await task)
    return results

if __name__ == '__main__':

    print("Downloading Ratings...")
    ratings = pd.read_csv(os.environ.get('RATINGS_PATH'))
    parallel_results = asyncio.run(fill_df_async(ratings, genre_dict_create()))

    print(len(parallel_results))

ModuleNotFoundError: No module named 'numpy.rec'

In [24]:
print("Downloading Ratings...")
ratings = pd.read_csv(os.environ.get('RATINGS_PATH'))
results = asyncio.run(fill_df_async(ratings, genre_dict_create()))

# Convert results to a DataFrame (excluding 'not_found' entries)
movie_df = pd.DataFrame([result for result in results if isinstance(result, dict)])

display(movie_df.head())

Processing Movies: 100%|██████████| 1018/1018 [00:02<00:00, 378.99it/s]


,Movie_ID,Title,Genres,Language,Overview,Popularity,Release_Date,Vote_Avg,Budget,Revenue,Runtime,Keywords,Directors,Producers,Top_Cast,Top_Cast_Popularity_Avg
0,250574,Creep,[Horror],en,"Looking for work, Aaron comes across a cryptic...",9.185,2014-06-23,6.400,0,0,82,"[psychopath, bath, mountain, stalker, woods, d...",[Patrick Brice],"[Jason Blum, Mark Duplass]","[Mark Duplass, Patrick Brice, Katie Aselton]",3.732667
1,318846,The Big Short,"[Comedy, Drama]",en,The men who made millions from a global econom...,25.403,2015-12-11,7.400,28000000,133346506,131,"[based on novel or book, bank, fraud, wall str...",[Adam McKay],"[Jeremy Kleiner, Arnon Milchan, Brad Pitt, Ded...","[Steve Carell, Christian Bale, John Magaro, Ry...",12.632500
2,862965,Emily the Criminal,"[Crime, Drama, Thriller]",en,"Desperate for income, Emily takes a shady gig ...",11.649,2022-08-12,6.681,2000000,2157673,97,"[job interview, organized crime, los angeles, ...",[John Patton Ford],"[Aubrey Plaza, Tyler Davidson, Drew Sykes]","[Aubrey Plaza, Theo Rossi, Megalyn Echikunwoke...",8.001500
3,37735,Easy A,[Comedy],en,"Olive, an average high school student, sees he...",12.630,2010-09-16,6.800,8000000,75026327,93,"[high school, small town, lie, loss of virgini...",[Will Gluck],"[Zanne Devine, Will Gluck]","[Emma Stone, Penn Badgley, Amanda Bynes, Dan B...",9.562700
4,212778,Chef,[Comedy],en,When Chef Carl Casper suddenly quits his job a...,14.683,2014-05-08,7.100,11000000,45967935,114,"[parent child relationship, restaurant owner, ...",[Jon Favreau],"[Sergey Bespalov, Jon Favreau]","[Jon Favreau, John Leguizamo, Bobby Cannavale,...",20.653200
